# Importing the libaries

In [1]:
!pip install ensemble_boxes

In [2]:
!pip install path

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rcParams

%matplotlib inline

import os
import cv2
import random
import shutil
from tqdm.auto import tqdm
from sklearn.model_selection import train_test_split
from ensemble_boxes import weighted_boxes_fusion

# Mounting the drive

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Loading the train annotations data

In [5]:
train_annotations = pd.read_csv("/content/drive/MyDrive/VinDr/annotations/annotations_train.csv")

In [6]:
train_annotations.head()

,image_id,rad_id,class_name,x_min,y_min,x_max,y_max
0,000434271f63a053c4128a0ba6352c7f,R2,No finding,NaN,NaN,NaN,NaN
1,000434271f63a053c4128a0ba6352c7f,R3,No finding,NaN,NaN,NaN,NaN
2,000434271f63a053c4128a0ba6352c7f,R6,No finding,NaN,NaN,NaN,NaN
3,00053190460d56c53cc3e57321387478,R11,No finding,NaN,NaN,NaN,NaN
4,00053190460d56c53cc3e57321387478,R2,No finding,NaN,NaN,NaN,NaN


In [8]:
train_annotations.tail()

,image_id,rad_id,class_name,x_min,y_min,x_max,y_max
69047,fff0f82159f9083f3dd1f8967fc54f6a,R8,No finding,NaN,NaN,NaN,NaN
69048,fff0f82159f9083f3dd1f8967fc54f6a,R9,No finding,NaN,NaN,NaN,NaN
69049,fff2025e3c1d6970a8a6ee0404ac6940,R1,No finding,NaN,NaN,NaN,NaN
69050,fff2025e3c1d6970a8a6ee0404ac6940,R2,No finding,NaN,NaN,NaN,NaN
69051,fff2025e3c1d6970a8a6ee0404ac6940,R5,No finding,NaN,NaN,NaN,NaN


In [7]:
train_annotations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69052 entries, 0 to 69051
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   image_id    69052 non-null  object 
 1   rad_id      69052 non-null  object 
 2   class_name  69052 non-null  object 
 3   x_min       37367 non-null  float64
 4   y_min       37367 non-null  float64
 5   x_max       37367 non-null  float64
 6   y_max       37367 non-null  float64
dtypes: float64(4), object(3)
memory usage: 3.7+ MB


# Function to calculate Intersection over Union (IoU)

In [9]:
def iou(box1, box2):
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])

    intersection = max(0, x2 - x1) * max(0, y2 - y1)
    area1 = (box1[2] - box1[0]) * (box1[3] - box1[1])
    area2 = (box2[2] - box2[0]) * (box2[3] - box2[1])
    union = area1 + area2 - intersection

    return intersection / union if union > 0 else 0

# Function to fuse bounding boxes

In [10]:
def weighted_fuse(boxes, iou_threshold=0.5):
    """
    Fuse bounding boxes:
    - If IoU is above the threshold, retain only the largest box.
    - If there's only one box, return it as is.
    """
    fused_boxes = []

    while boxes:
        base_box = boxes.pop(0)
        to_merge = [base_box]

        # Find all boxes that overlap with the base_box
        for box in boxes[:]:
            if iou(base_box, box) >= iou_threshold:
                to_merge.append(box)
                boxes.remove(box)

        # If there are overlapping boxes, retain only the largest one
        if len(to_merge) > 1:
            largest_box = max(to_merge, key=lambda b: (b[2] - b[0]) * (b[3] - b[1]))  # Area calculation
            fused_boxes.append(largest_box)
        else:
            # If no significant overlaps, just add the single box
            fused_boxes.append(to_merge[0])

    return fused_boxes

# Function to process annotations to fuse bounding boxes

In [11]:
def grouping_annotations(df, iou_threshold=0.5):
    # Drop rows where there are no bounding boxes (e.g., 'No finding')
    df = df.dropna(subset=['x_min', 'y_min', 'x_max', 'y_max'])

    # Group data by image_id and class_name
    grouped = df.groupby(['image_id', 'class_name'])
    fused_results = []

    for (image_id, class_name), group in grouped:

        boxes = group[['x_min', 'y_min', 'x_max', 'y_max']].values.tolist()
        fused_boxes = weighted_fuse(boxes, iou_threshold=iou_threshold)

        for box in fused_boxes:
            fused_results.append({
                "image_id": image_id,
                "class_name": class_name,
                "x_min": box[0],
                "y_min": box[1],
                "x_max": box[2],
                "y_max": box[3],
            })

    return pd.DataFrame(fused_results)

# Function to process annotations

In [12]:
def process_annotations(df, iou_threshold=0.5):
    fused_data = grouping_annotations(df, iou_threshold=iou_threshold)

    # For training annotations, handle "No finding" separately
    if "rad_id" in df.columns:
        no_finding_images = (
            df[df["class_name"] == "No finding"]
            .groupby("image_id")
            .filter(lambda x: len(x) == 3)
            .drop_duplicates(subset="image_id")
        )
        no_finding_image_ids = set(no_finding_images["image_id"].unique())
        fused_image_ids = set(fused_data["image_id"].unique())
        missing_no_finding_ids = no_finding_image_ids - fused_image_ids
        missing_no_finding_rows = df[
            (df["image_id"].isin(missing_no_finding_ids)) &
            (df["class_name"] == "No finding")
        ].drop_duplicates(subset="image_id")
        fused_data = pd.concat([fused_data, missing_no_finding_rows], ignore_index=True)

    return fused_data

    # # Step 1: Identify images where all three radiologists marked "No findings"
    # no_finding_images = (
    #     df[df["class_name"] == "No finding"]
    #     .groupby("image_id")
    #     .filter(lambda x: len(x) == 3)  # Ensure all 3 radiologists have marked "No findings"
    #     .drop_duplicates(subset="image_id")  # Ensure one instance per image
    # )

    # # Step 2: Extract unique image_ids from the identified rows
    # no_finding_image_ids = set(no_finding_images["image_id"].unique())

    # # Step 3: Extract image_ids from fused_data
    # fused_image_ids = set(fused_data["image_id"].unique())

    # # Step 4: Identify images missing in fused_data
    # missing_no_finding_ids = no_finding_image_ids - fused_image_ids

    # # Step 5: Append rows for missing image_ids to fused_data
    # missing_no_finding_rows = df[
    #     (df["image_id"].isin(missing_no_finding_ids)) &
    #     (df["class_name"] == "No finding")
    # ].drop_duplicates(subset="image_id")

    # # Append the missing rows to fused_data
    # fused_data = pd.concat([fused_data, missing_no_finding_rows], ignore_index=True)
    # return fused_data

In [13]:
# Process train annotations
processed_train_annotations = process_annotations(train_annotations, iou_threshold=0.5)

In [14]:
# Adding class_id to train annotations
class_mapping = {
    "No finding": 0, "Infiltration": 1, "Lung Opacity": 2, "Consolidation": 3,
    "Nodule/Mass": 4, "Pulmonary fibrosis": 5, "Pleural thickening": 6,
    "Aortic enlargement": 7, "Cardiomegaly": 8, "ILD": 9, "Other lesion": 10,
    "Pleural effusion": 11, "Calcification": 12, "Enlarged PA": 13,
    "Lung cavity": 14, "Atelectasis": 15, "Mediastinal shift": 16,
    "Lung cyst": 17, "Pneumothorax": 18, "Emphysema": 19,
    "Clavicle fracture": 20, "Rib fracture": 21, "Edema": 22
}

In [15]:
processed_train_annotations["class_id"] = processed_train_annotations["class_name"].map(class_mapping)

In [ ]:
# Save processed train annotations
output_folder_path = "/content/drive/MyDrive/VinDr/annotations/"
os.makedirs(output_folder_path, exist_ok=True)
processed_train_annotations.to_csv(os.path.join(output_folder_path, "processed_train_annotations.csv"), index=False)

# Applying same processing to test data


In [16]:
# Load and process test annotations
test_annotations = pd.read_csv("/content/drive/MyDrive/VinDr/annotations/annotations_test.csv")

In [17]:
test_annotations.shape

(4748, 6)

In [18]:
# Since there is no rad_id, we directly fuse bounding boxes
processed_test_annotations = grouping_annotations(test_annotations, iou_threshold=0.5)
processed_test_annotations["class_id"] = processed_test_annotations["class_name"].map(class_mapping)

In [ ]:
# Save processed test annotations
processed_test_annotations.to_csv(os.path.join(output_folder_path, "processed_test_annotations.csv"), index=False)

In [19]:
processed_train_annotations.drop(columns=['class_id'], inplace=True)

In [20]:
processed_train_annotations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35624 entries, 0 to 35623
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   image_id    35624 non-null  object 
 1   class_name  35624 non-null  object 
 2   x_min       25146 non-null  float64
 3   y_min       25146 non-null  float64
 4   x_max       25146 non-null  float64
 5   y_max       25146 non-null  float64
 6   rad_id      10478 non-null  object 
dtypes: float64(4), object(3)
memory usage: 1.9+ MB


In [21]:
train_annotations['class_name'].unique()

array(['No finding', 'Infiltration', 'Lung Opacity', 'Consolidation',
       'Nodule/Mass', 'Pulmonary fibrosis', 'Pleural thickening',
       'Aortic enlargement', 'Cardiomegaly', 'ILD', 'Other lesion',
       'Pleural effusion', 'Calcification', 'Enlarged PA', 'Lung cavity',
       'Atelectasis', 'Mediastinal shift', 'Lung cyst', 'Pneumothorax',
       'Emphysema', 'Clavicle fracture', 'Rib fracture', 'Edema'],
      dtype=object)

In [ ]:
for i in train_annotations['class_name'].unique():
  print(f'Counts for abnormality {i} in the datasets:')
  print("Before weighted fusion")
  print(train_annotations[train_annotations['class_name']== i]['image_id'].count())
  print("After weighted fusion")
  print(processed_train_annotations[processed_train_annotations['class_name']== i]['image_id'].count())
  print('--' * 5)

Counts for abnormality No finding in the datasets:
Before weighted fusion
31685
After weighted fusion
10478
----------
Counts for abnormality Infiltration in the datasets:
Before weighted fusion
1247
After weighted fusion
961
----------
Counts for abnormality Lung Opacity in the datasets:
Before weighted fusion
2493
After weighted fusion
2033
----------
Counts for abnormality Consolidation in the datasets:
Before weighted fusion
556
After weighted fusion
435
----------
Counts for abnormality Nodule/Mass in the datasets:
Before weighted fusion
2611
After weighted fusion
1892
----------
Counts for abnormality Pulmonary fibrosis in the datasets:
Before weighted fusion
4659
After weighted fusion
3387
----------
Counts for abnormality Pleural thickening in the datasets:
Before weighted fusion
4884
After weighted fusion
4119
----------
Counts for abnormality Aortic enlargement in the datasets:
Before weighted fusion
7193
After weighted fusion
3484
----------
Counts for abnormality Cardiomega

In [22]:
processed_train_annotations['image_path'] = processed_train_annotations['image_id'].map(lambda x:os.path.join('/content/drive/MyDrive/100 images data/converted_images1', str(x)+'.png'))
processed_train_annotations['image_path'] = processed_train_annotations['image_id'].map(lambda x:os.path.join('/content/drive/MyDrive/100 images data/converted_images1', str(x)+'.png'))
processed_train_annotations.head(10)

,image_id,class_name,x_min,y_min,x_max,y_max,rad_id,image_path
0,0005e8e3701dfb1dd93d53e2ff537b6e,Consolidation,932.471985,567.778992,1197.770020,896.408997,NaN,/content/drive/MyDrive/100 images data/convert...
1,0005e8e3701dfb1dd93d53e2ff537b6e,Infiltration,900.956970,587.809021,1205.359985,888.710999,NaN,/content/drive/MyDrive/100 images data/convert...
2,0005e8e3701dfb1dd93d53e2ff537b6e,Lung Opacity,900.956970,587.809021,1205.359985,888.710999,NaN,/content/drive/MyDrive/100 images data/convert...
3,0005e8e3701dfb1dd93d53e2ff537b6e,Nodule/Mass,932.471985,567.778992,1197.770020,896.408997,NaN,/content/drive/MyDrive/100 images data/convert...
4,0007d316f756b3fa0baea2ff514ce945,Aortic enlargement,1235.979980,1021.640015,1482.900024,1281.979980,NaN,/content/drive/MyDrive/100 images data/convert...
5,0007d316f756b3fa0baea2ff514ce945,Cardiomegaly,902.039978,1827.739990,1829.670044,2037.020020,NaN,/content/drive/MyDrive/100 images data/convert...
6,0007d316f756b3fa0baea2ff514ce945,ILD,1847.310059,1409.989990,2093.120117,2096.550049,NaN,/content/drive/MyDrive/100 images data/convert...
7,0007d316f756b3fa0baea2ff514ce945,ILD,535.403992,1748.979980,932.219971,2013.060059,NaN,/content/drive/MyDrive/100 images data/convert...
8,0007d316f756b3fa0baea2ff514ce945,Pleural thickening,818.666016,677.098022,987.669983,939.344971,NaN,/content/drive/MyDrive/100 images data/convert...
9,0007d316f756b3fa0baea2ff514ce945,Pleural thickening,621.364990,673.406006,1025.859985,851.734009,NaN,/content/drive/MyDrive/100 images data/convert...


In [ ]:
def plot_img(img, size=(8, 8), is_rgb=True, title="", cmap='gray'):
    """
    Plots a single image with customizable size and title.

    Parameters:
    - img: Image to be displayed.
    - size: Tuple for figure size.
    - is_rgb: Boolean to specify if the image is RGB.
    - title: Title for the plot.
    - cmap: Colormap for the plot.
    """
    plt.figure(figsize=size)
    if is_rgb:
        plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    else:
        plt.imshow(img, cmap=cmap)
    plt.title(title, fontsize=16)
    plt.axis('off')
    plt.show()


def plot_imgs(imgs, cols=2, size=5, is_rgb=True, title="", cmap='gray', img_size=None):
    """
    Plots multiple images in a grid.

    Parameters:
    - imgs: List of images.
    - cols: Number of columns.
    - size: Size of each subplot.
    - is_rgb: Boolean to specify if the images are RGB.
    - title: Title for the entire plot.
    - cmap: Colormap for the plot.
    - img_size: Desired size for resizing images.
    """
    rows = len(imgs) // cols + (len(imgs) % cols > 0)
    fig, axes = plt.subplots(rows, cols, figsize=(cols * size, rows * size))
    axes = axes.flatten()

    for i, img in enumerate(imgs):
        if img_size is not None:
            img = cv2.resize(img, img_size)
        if is_rgb:
            axes[i].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        else:
            axes[i].imshow(img, cmap=cmap)
        axes[i].axis('off')

    for j in range(len(imgs), len(axes)):
        axes[j].axis('off')  # Hide unused subplots

    plt.suptitle(title, fontsize=20)
    plt.tight_layout()
    plt.show()


def draw_bbox(image, box, label, color, thickness=2, font_scale=1.0):
    """
    Draws a bounding box with a label on an image.

    Parameters:
    - image: The image to draw on.
    - box: List or tuple [x_min, y_min, x_max, y_max].
    - label: Text label for the box.
    - color: Color of the box and label background.
    - thickness: Thickness of the bounding box lines.
    - font_scale: Scale for the label text.

    Returns:
    - output: Image with the bounding box and label.
    """
    output = image.copy()

    # Draw the bounding box (only the border)
    cv2.rectangle(output, (box[0], box[1]), (box[2], box[3]), color, thickness)

    # Draw the label background
    label_size, _ = cv2.getTextSize(label.upper(), cv2.FONT_HERSHEY_SIMPLEX, font_scale, 2)
    label_width, label_height = label_size
    label_bg_start = (box[0], box[1] - label_height - 5)
    label_bg_end = (box[0] + label_width + 10, box[1])
    cv2.rectangle(output, label_bg_start, label_bg_end, color, -1)

    # Add the label text
    label_pos = (box[0] + 5, box[1] - 5)
    cv2.putText(output, label.upper(), label_pos, cv2.FONT_HERSHEY_SIMPLEX, font_scale, (255, 255, 255), 2, cv2.LINE_AA)

    return output

In [ ]:
# Ensure viz_labels are properly defined
viz_labels = [
    "No finding", "Infiltration", "Lung Opacity", "Consolidation", "Nodule/Mass",
    "Pulmonary fibrosis", "Pleural thickening", "Aortic enlargement", "Cardiomegaly",
    "ILD", "Other lesion", "Pleural effusion", "Calcification", "Enlarged PA",
    "Lung cavity", "Atelectasis", "Mediastinal shift", "Lung cyst", "Pneumothorax",
    "Emphysema", "Clavicle fracture", "Rib fracture", "Edema"
]

# Mapping label_id to color (23 unique colors for the abnormalities)
label2color = [
    [59, 238, 119], [222, 21, 229], [94, 49, 164], [206, 221, 133], [117, 75, 3],
    [210, 224, 119], [211, 176, 166], [63, 7, 197], [102, 65, 77], [194, 134, 175],
    [209, 219, 50], [255, 44, 47], [89, 125, 149], [110, 27, 100], [158, 200, 98],
    [48, 190, 123], [155, 104, 180], [85, 240, 232], [142, 68, 173], [244, 208, 63],
    [241, 90, 34], [52, 152, 219], [46, 204, 113]
]

In [ ]:
from path import Path
# Assuming `train_annotations`, `processed_train_annotations` are pre-loaded pandas DataFrames
viz_images = []
copy_data = train_annotations.copy()
copy_data = copy_data[copy_data['class_id'] != 0]
processed_copy_data = processed_train_annotations.copy()
processed_copy_data = processed_copy_data[processed_copy_data['class_id'] != 0]

# Ensure you have the correct 'image_path' column and 'image_id' column
for i, path in tqdm(enumerate(copy_data['image_path'].unique()[15:18])):
    img_array = cv2.imread(path)
    image_basename = Path(path).stem
    print(f"(\'{image_basename}\', \'{path}\')")
    img_annotations = copy_data[copy_data.image_id == image_basename]

    boxes_viz = img_annotations[['x_min', 'y_min', 'x_max', 'y_max']].to_numpy().tolist()
    labels_viz = img_annotations['class_id'].to_numpy().tolist()

    # Visualize Original Bboxes
    img_before = img_array.copy()
    for box, label in zip(boxes_viz, labels_viz):
        x_min, y_min, x_max, y_max = box
        color = label2color[int(label)]
        img_before = draw_bbox(img_before, list(np.int_(box)), viz_labels[label], color)
    viz_images.append(img_before)

    processed_img_annotations = processed_copy_data[processed_copy_data.image_id == image_basename]

    processed_boxes_viz = processed_img_annotations[['x_min', 'y_min', 'x_max', 'y_max']].to_numpy().tolist()
    processed_labels_viz = processed_img_annotations['class_id'].to_numpy().tolist()

    # Visualize Processed Bboxes
    img_after = img_array.copy()
    for box, label in zip(processed_boxes_viz, processed_labels_viz):
        x_min, y_min, x_max, y_max = box
        color = label2color[int(label)]
        img_after = draw_bbox(img_after, list(np.int_(box)), viz_labels[label], color)
    viz_images.append(img_after)

# Plot the images with a comparison of original and processed bounding boxes
plot_imgs(viz_images, cmap=None)
plt.figtext(0.3, 0.9, "Original Bboxes", va="top", ha="center", size=25)
plt.figtext(0.73, 0.9, "Processed Bboxes", va="top", ha="center", size=25)
plt.show()

0it [00:00, ?it/s]

('010018c93ed33ae56ed048ee54867e46', '/content/drive/MyDrive/100 images data/converted_images1/010018c93ed33ae56ed048ee54867e46.png')
('0108949daa13dc94634a7d650a05c0bb', '/content/drive/MyDrive/100 images data/converted_images1/0108949daa13dc94634a7d650a05c0bb.png')


IndexError: list index out of range

In [ ]:
processed_train_annotations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 280 entries, 0 to 279
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   image_id    280 non-null    object 
 1   class_name  280 non-null    object 
 2   x_min       213 non-null    float64
 3   y_min       213 non-null    float64
 4   x_max       213 non-null    float64
 5   y_max       213 non-null    float64
 6   rad_id      67 non-null     object 
 7   class_id    280 non-null    int64  
 8   image_path  280 non-null    object 
dtypes: float64(4), int64(1), object(4)
memory usage: 19.8+ KB


In [ ]:
len(label2color)

14

In [ ]:
train_annotations['class_name'].nunique()

18